# TOUGH3 preprocessing v1.1

- Create and read already generated meshes for TOUGH3 with meshmaker
- Set all necessary input parameters and initial conditions for INFILE and INCON generation

In [1]:
# Import the required libraries
import toughio
import itertools
import numpy as np
import pandas as pd

In [2]:
# read MESH

def get_thicknesses(center_pts, x_init, x_end):
    """
    Function to get the cell thicknesses from the center points for one axis coordinates
    """
    thick_array = np.zeros_like(center_pts)
    for i in range(len(center_pts)):
        ht = center_pts[i] - x_init
        x_cell_max = center_pts[i] + ht
        thick_array[i] = x_cell_max - x_init
        x_init = x_cell_max
    print('------------------------------------------------------------')
    print('The sum of all thicknesses is: ' + str(np.sum(thick_array)), )
    print('The difference to the aimed section length/height is: ' + str(np.abs(np.sum(thick_array) - x_end)), )
    return thick_array

input_path = '/home/victor/Desktop/toughio/examples/heat_and_fluid_flow_from_magmatic_fluids/'

output_path = '/home/victor/Desktop/toughio/examples/heat_and_fluid_flow_from_magmatic_fluids/2_injection/'

mesh = toughio.read_mesh(input_path + 'Antonios_MESH')

cell_ids = list(mesh['elements'].keys())

x_vals = []
z_vals = []
for i in range(len(cell_ids)):
    x_vals.append(mesh['elements'][cell_ids[i]]['center'][0])
    z_vals.append(mesh['elements'][cell_ids[i]]['center'][2])
x_centers = np.unique(np.asarray(x_vals))
z_centers = np.flip(np.unique(np.asarray(z_vals))[0:-1]) * (-1)

x_thick = get_thicknesses(x_centers, x_init=0.0, x_end=10000.0)
x_rad = np.insert(np.asarray(list(itertools.accumulate(x_thick))), 0, 0.0)
z_thick = get_thicknesses(z_centers, x_init=0.0, x_end=1500.0)

# add cells at the top for boundary conditions
z_thick = np.insert(np.flip(z_thick), len(z_thick), 0.01)

parameters = {
    'meshmaker': {
        'type': 'rz2dl',
        'parameters': [
            {'type': 'radii', 'radii': list(x_rad)},
            {'type': 'layer', 'thicknesses': list(z_thick)}
        ]
    }
}

mesh = toughio.meshmaker.from_meshmaker(parameters, material='HROCK')

# prepare different rock type for setting boundary conditions
ind = np.where(mesh.centers[:, 2] >= -2.0)[0]
materials_new = np.ones((len(mesh.materials)))
materials_new[ind] = 2
# overwrite existing material names ...
mesh.add_cell_data('material', materials_new.astype(int))
# rename materials
mesh.add_material('HROCK', 1)
mesh.add_material('BOUND', 2)

# print the number of elements of the computational domain
print('Number of elements: ' + str(len(mesh.centers)))

materials = mesh.materials
bcond = (materials == 'BOUND').astype(int)
mesh.add_cell_data('boundary_condition', bcond)

# initial conditions (single-phase)
# get the last index of the boundary
bound_idx = np.where(mesh.materials=='BOUND')[0][-1] + 1
incon = np.full((mesh.n_cells, 3), -1.0e9)
incon[:bound_idx, 0] = 1.0132e5                                                     # atmospheric conditions for pressure
incon[bound_idx:, 0] = 1.0132e5 - 997 * 9.81 * mesh.centers[bound_idx:, 2]          # hydrostatic pore pressure
incon[:bound_idx, 1] = 21.0                                                         # temperature at boundary conditions
# incon[bound_idx:, 1] = 21.0 - 0.0267 * mesh.centers[bound_idx:, 2]                # some temperature gradient
incon[bound_idx:, 1] = 21.0 - 0.219 * mesh.centers[bound_idx:, 2]                   # some temperature gradient
# incon[bound_idx:, 1] = 180.0
incon[:bound_idx, 2] = 35.0                                                         # co2 partial pressure at the boundary
incon[bound_idx:, 2] = 35.0 - 0.0034 * mesh.centers[bound_idx:, 2]                                                         # co2 partial pressure at the rock
mesh.add_cell_data('initial_condition', incon)

mesh.write_tough(output_path + 'MESH', incon=True)
mesh.write(output_path + 'mesh.pickle')

------------------------------------------------------------
The sum of all thicknesses is: 10000.6
The difference to the aimed section length/height is: 0.6000000000003638
------------------------------------------------------------
The sum of all thicknesses is: 1500.0
The difference to the aimed section length/height is: 0.0
Number of elements: 2580


In [4]:
# Initialize parameters dictionary

parameters = {
    'title': 'Heat and fluid flow simulation from magmatic fluid intrusion at depth',
    'eos': 'eos2',              # eos2 to inject co2 and water
    'n_component': 2,           # number of components
    'n_phase': 2,               # number of phases
    'start': True,
    'isothermal': False,        # temperature changes during the simulation
}

# Define the properties of the materials in the mesh
parameters['default'] = {
    'density': 2000.0,
    'porosity': 0.2,
    'conductivity': 2.8,
    'specific_heat': 1000.0,
}

# Define initial conditions
parameters['default'] = {
    'initial_condition': [1.0132e5, 21.0, 35.0],       # pressure, temperature, CO2 partial pressure (if zero, only water is present!)
}

parameters['rocks'] = {
    'HROCK': {
        'density': 2000.0,
        'porosity': 0.2,
        'permeability': 1.0e-14,
        'conductivity': 2.8,
        'specific_heat': 1000.0,
        # 'initial_condition': [1.0132e5, 180.0, 35.0],       # pressure, temperature, CO2 partial pressure
    },
    'BOUND': {
        'density': 2000.0,
        'porosity': 0.2,
        'permeability': 1.0e-13,
        'conductivity': 2.8,
        'specific_heat': 1.0e55,
    #     'initial_condition': [1.0132e5, 21.0, 35.0],        # pressure, temperature, CO2 partial pressure
    }
}

# Specify the simulation parameters (block 'PARAM')
parameters['options'] = {
    'n_cycle': 9999,                                    # maximum number of time steps
    'n_cycle_print': 9999,                              # printout for every multiple of this number (if 9999, printout as specified in parameters times (see cell bottom)
    't_ini': 0.0,                                       # starting time of the simulation
    't_max': 7000.0 * 365.25 * 24.0 * 3600.0,           # time in seconds at which the simulation stops
    # 't_steps': 10.0 * 365.25 * 24.0 * 3600.0,
    # 't_steps': -1.0,
    't_steps': 1.0e4,                                   # time step size
    't_step_max': 95.0 * 365.25 * 24.0 * 3600.0,        # upper limit for time step size
    # 't_reduce_factor': 4,
    'eps1': 1.0e-5,                                     # convergence criterion for relative error
    'gravity': 9.81,
    'initial_conditions': [1.0132e5, 21.0, 35.0],       # initial conditions (as defined above)
}

# Select choice of various options for block 'MOP'
parameters['extra_options'] = {
    1: 1,   # printout for non-convergent iterations
    7: 1,   # printout of input data provided
    13: 1,  # writes user-specific initial conditions to file SAVE
    16: 4,  # time step size will be doubled if convergence occurs within ITER <= 4 Newton-Raphson iterations
    18: 0,  # performs upstream weighting for interface density
    21: 3,  # DSLUCS linear equation solver
}

# Define choice of relative permeability and capillary pressure functions
parameters['default'] = {
    'relative_permeability': {
        'id': 3,
        'parameters': [0.33, 0.05],         # check paper to confirm
    },
    'capillarity': {
        'id': 1,
        'parameters': [0.0, 0.0, 1.0],      # this is not entirely correct yet
    },
}

# Now we define the generators (i.e., the sources)
# The injection extent is of 150 m. We need to constrain the injected into the first cell (1) or we look for the cells that cover those 150 m and inject into them (2)
inj_opt = '2'   # option 1 is not yet working quite well ...

parameters['generators'] = []

inj_max = 150.0
comp = ['COM1', 'COM2']
com1_inj = 2400.0 * 1000.0 / (24.0 * 60.0 * 60.0)       # injected component 1 (water)
com2_inj = 1000.0 * 1000.0 / (24.0 * 60.0 * 60.0)       # injected component 2 (co2)
enthalpy = [2.58e6, 6.83e5]                             # enthalpies
# to check the enthalpies: https://irc.wisc.edu/properties/

if inj_opt=='2':
    # We need to look for all cells and distribute the injection rate according to their respective volumetric contributions to the domain.
    # Let's unpickle the mesh back and use the method :meth: 'toughio.Mesh.near' to get the name of the injection element.
    mesh = toughio.read_mesh(output_path + 'mesh.pickle')
    label_1 = mesh.labels[mesh.near((0.0, 0.0, -1500.0))]
    ind_1 = int(np.where(mesh.labels == label_1)[0])
    label_2 = mesh.labels[mesh.near((inj_max, 0.0, -1500.0))]
    ind_2 = int(np.where(mesh.labels == label_2)[0]) + 1  # add one indice more
    # get lists/arrays of all generators
    label_all = mesh.labels[ind_1:ind_2]
    # way 1 #
    vols = mesh.volumes[ind_1:ind_2] / np.sum(mesh.volumes[ind_1:ind_2])
    # introduce an error if sum(vols)!=1.0 #
    # injection rates
    rates_all = [com1_inj * vols, com2_inj * vols]

    for i in range(len(comp)):
        for j in range(len(label_all)):
            parameters['generators'].append(
                {
                    'label': label_all[j],
                    'type': comp[i],
                    'rates': rates_all[i][j],
                    'specific_enthalpy': enthalpy[i],
                }
            )
else:
    mesh = toughio.read_mesh(output_path + 'mesh.pickle')
    label = mesh.labels[mesh.near((0.0, 0.0, -1500.0))]
    V_tot = inj_max**2 * np.pi * z_thick[0]
    V_inj_cell = x_thick[0]**2 * np.pi * z_thick[0]
    ratio = V_inj_cell / V_tot
    rates_all = [com1_inj, com2_inj] / (1/ratio)

    for i in range(len(comp)):
        parameters['generators'].append(
            {
                'label': label,
                'type': comp[i],
                'rates': rates_all[i],
                'specific_enthalpy': enthalpy[i],
            }
        )

# Customize the outputs
years_range = [1.0, 5.0, 10.0, 25.0, 50.0, 100.0, 150.0, 250.0, 500.0, 750.0, 1000.0, 2000.0, 2500.0, 5000.0, 7000.0]
y = 365.25 * 24.0 * 3600.0
parameters['times'] = [x * y for x in years_range]

# Print only the following listed variables ...
parameters['output'] = {
    'variables': [
        {'name': 'saturation'},
        {'name': 'temperature'},
        {'name': 'pressure'},
        {'name': 'coordinate'},
    ]
}

toughio.write_input(output_path + 'INFILE', parameters)

/tmp/ipykernel_15195/773547481.py:100: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  ind_1 = int(np.where(mesh.labels == label_1)[0])
/tmp/ipykernel_15195/773547481.py:102: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  ind_2 = int(np.where(mesh.labels == label_2)[0]) + 1  # add one indice more
